In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
from scipy.spatial.transform import Rotation as R  # rotation axis ??? left-hand / clockwise

from base import Camera
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches

np.set_printoptions(suppress=True)

In [36]:
def _find(l, s):
    for elem in l:
        if elem['id'] == s or elem['id'] == f'{s}_0':
            values = elem['values']
            for i, value in enumerate(values):
                if value['labelName'] == 'cleaning_snuggle_henkel':
                    break
            else:
                return None
            return values[i]


folder_dir = './data/solo_basic'

for i in range(3):
    f_dir = f'{folder_dir}/sequence.{i}'
    with open(f'{f_dir}/step0.frame_data.json') as json_file:
        data = json.load(json_file)
    cameras = data['captures']

    for camera_info in cameras:
        # print('camera position', camera_info['position'])

        camera = Camera(
            position=camera_info['position'],
            quaternion=camera_info['rotation'],
            resolution=camera_info['dimension'],
        )

        # raster_coor = camera.world_to_camera(
        #     [
        #         np.array([0, 0, 15]),
        #         # np.array([2, 1.5, 2]),
        #     ]
        # )

        bbox_2d = _find(camera_info['annotations'], 'bounding box')
        center, size = bbox_2d['origin'], np.array([bbox/2 for bbox in bbox_2d['dimension']])
        # fig, ax = plt.subplots()
        # ax.imshow(mpimg.imread(f'{folder_dir}/{camera_info["filename"]}'))
        # ax.add_patch(
        #     patches.Rectangle(
        #         center,
        #         # raster_coor[0],
        #         2*size[0], 2*size[1],
        #         linewidth=1, edgecolor='r', facecolor='none'
        #     )
        # )
        # plt.plot(
        #     raster_coor[0][0], raster_coor[0][1],
        #     "o", markersize=5, markeredgecolor="blue", markerfacecolor="green"
        # )
        # plt.axis('off')
        # plt.show()
        print(center + size/2)
        # print(center + np.array(size)/2, np.array(size))
        # print(raster_coor)

        # print(raster_coor[0])


[1920. 1081.]
[2453. 1081.]
[1920.  548.]


In [62]:
camera_coor = [
    np.array([0, 0, 15]),
    np.array([3, 0, 15]),
    np.array([0, 3, 15]),
]

focal = 20.78461
resolution = np.array([3840, 2160])
sensor = np.array([30, 30])
intrinsic = np.array([
    # [focal*resolution[0]/sensor[0], 0, resolution[0]/2, 0],
    # [0, focal*resolution[1]/sensor[1], resolution[1]/2, 0],
    # [0, 0, 1, 0],
    ###
    # [focal*resolution[0]/sensor[0], 0, resolution[0]/2],
    # [0, focal*resolution[1]/sensor[1], resolution[1]/2],
    # [0, 0, 1],
    ###
    # [focal*resolution[1]/sensor[0], 0, 0],
    # [0, -focal*resolution[1]/sensor[1], 0],
    # [0, 0, 1],
    ###
    [focal*resolution[0]/sensor[0], 0, 0],
    [0, -focal*resolution[1]/sensor[1], 0],
    [resolution[0]/2, resolution[1]/2, 1],
])
ndc_coor = [np.matmul(coor, intrinsic) for coor in camera_coor]

pixel_coor = [(coor/coor[-1])[:-1] for coor in ndc_coor]
# pixel_coor = [ 
#     np.array([
#         coor[0] + resolution[0]/2,
#         coor[1] + resolution[1]/2,
#     ])
#     for coor in ndc_coor
# ]
for coor in pixel_coor:
    print(coor)

[1920. 1080.]
[2219.298384 1080.      ]
[1920.        780.701616]


In [ ]:
def qm(quaternion1, quaternion0):
    w0, x0, y0, z0 = quaternion0
    w1, x1, y1, z1 = quaternion1
    
    return np.array([
        -x1*x0 - y1*y0 - z1*z0 + w1*w0,
         x1*w0 - y1*z0 + z1*y0 + w1*x0,
         x1*z0 + y1*w0 - z1*x0 + w1*y0,
        -x1*y0 + y1*x0 + z1*w0 + w1*z0
    ])

q = np.array([0, 1, 0, 0])
abc = [
    np.array([0, 2, 1.5, 0]),
]
d = np.array([2, 2, 5])

_inv = np.array([1, -1, -1, -1])
print(
    qm(qm(q, abc[0]), q*_inv), qm(qm(q, abc[0]), q*_inv)[1:] - d
)

In [ ]:
abc = np.array([1, 2, 3])
np.insert(abc, 0, 0)